# Upload project

How to upload data to Supervisely using API

**Input**:
- Local folder with Project in Supervisely format

**Output**:
- New Project in Supervisely

## Configuration

Edit the following settings for your own case

In [1]:
import supervisely_lib as sly
import os
import json

In [3]:
# Local folder with Project
project_directory = './tutorial_project'

# New Project
team_name = "jupyter_tutorials"
workspace_name = "First Workspace"
project_name = "uploaded_project"

# Obtain server address and your api_token from environment variables
# Edit those values if you run this notebook on your own PC
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [4]:
# Initialize API object
api = sly.Api(address, token)

## Verify input values

Test that context (team / workspace / project) exists

In [5]:
# get IDs of team, workspace and project by names

team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))

## Read local Project

In [6]:
# read project from directory 
project_fs = sly.Project(project_directory, sly.OpenMode.READ)

## Create remote Project

In [7]:
# check if project already exists. If yes - generate new free name
if api.project.exists(workspace.id, project_name):
    project_name = api.project.get_free_name(workspace.id, project_name)

In [ ]:
# create remote project and set corresponding meta information
project = api.project.create(workspace.id, project_name)
api.project.update_meta(project.id, project_fs.meta.to_json())
print("Project: id={}, name={}".format(project.id, project.name))

In [9]:
# iterate over datasets, images and thier annotations in directory and upload that data to remote server
for dataset_fs in project_fs:
    dataset = api.dataset.create(project.id, dataset_fs.name)
    for item_name in dataset_fs:
        img_path, ann_path = dataset_fs.get_item_paths(item_name)
        
        #upload image if needed and add it to remote dataset 
        img_hash = sly.fs.get_file_hash(img_path)
        if api.image.check_image_uploaded(img_hash) is False:
            img_hash = api.image.upload_path(img_path)
        image = api.image.add(dataset.id, item_name, img_hash)
        
        #upload annotation to added image
        with open(ann_path) as f:
            ann_json = json.load(f)
        api.annotation.upload(image.id, ann_json)
        

In [10]:
print("Project {!r} has been sucessfully uploaded".format(project_name))
print("Number of uploaded images: ", api.project.get_images_count(project.id))

Project 'uploaded_project_002' has been sucessfully uploaded
Number of uploaded images:  5
